# Clustering Using Customer Transactions Data

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sn

import warnings
warnings.filterwarnings('ignore')

import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
retail_df = pd.read_csv("https://raw.githubusercontent.com/manaranjanp/ISB_MLUL/main/clustering/retail_txns.csv")

In [ ]:
retail_df.Country.value_counts()

In [ ]:
retail_df.info();

In [ ]:
retail_df.isnull().sum()

In [ ]:
retail_df.dropna( subset = ['CustomerID'], inplace = True)

In [ ]:
retail_df.info()

In [ ]:
retail_df.isnull().sum()

In [ ]:
retail_df.duplicated().sum()

In [ ]:
retail_df.drop_duplicates(inplace = True)

In [ ]:
retail_df.info();

In [ ]:
sn.kdeplot(retail_df.Quantity);

In [ ]:
retail_df[retail_df.Quantity < 0][0:10]

In [ ]:
sn.kdeplot(retail_df.UnitPrice);

### StockCode Information

In [ ]:
stock_df = retail_df[['CustomerID', 'StockCode']]

In [ ]:
stock_df.drop_duplicates(inplace = True)

In [ ]:
stock_df.info()

In [ ]:
len(stock_df.StockCode.unique())

In [ ]:
len(stock_df.CustomerID.unique())

In [ ]:
stock_count_df = stock_df.StockCode.value_counts().reset_index()
stock_count_df.columns = ['StockCode', 'count']

In [ ]:
plt.figure(figsize = (10, 5))
sn.histplot( stock_count_df['count']);

In [ ]:
stock_count_df = stock_count_df[stock_count_df['count'] > 10]
stock_count_df

### Customer Information

In [ ]:
cust_count_df = stock_df.CustomerID.value_counts().reset_index()
cust_count_df.columns = ['CustomerID', 'count']

In [ ]:
plt.figure(figsize = (10, 5))
sn.histplot( cust_count_df['count']);

In [ ]:
cust_count_df[cust_count_df['count'] > 10]
cust_count_df

In [ ]:
clean_stock_df = stock_df[stock_df.StockCode.isin(stock_count_df.StockCode.unique()) &
                          stock_df.CustomerID.isin(cust_count_df.CustomerID.unique())]

In [ ]:
clean_stock_df.info()

In [ ]:
clean_stock_df['bought'] = 1.0

In [ ]:
clean_stock_df

In [ ]:
cust_stocks_df = clean_stock_df.pivot( index='StockCode', 
                                       columns='CustomerID', 
                                       values = "bought" )
cust_stocks_df[0:10]

In [ ]:
cust_stocks_df.shape

In [ ]:
cust_stocks_df = cust_stocks_df.fillna(0.0)
cust_stocks_df.sample(10)

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
jaccard_distance = pairwise_distances(cust_stocks_df.values, metric="jaccard")

In [ ]:
jaccard_distance

In [ ]:
jaccard_distance.shape

In [ ]:
from sklearn.cluster import HDBSCAN

In [ ]:
h_cluster = HDBSCAN(min_cluster_size=2, metric = 'jaccard')

In [ ]:
#h_cluster = AgglomerativeClustering(n_clusters = 5)

In [ ]:
h_cluster.fit(cust_stocks_df)

In [ ]:
h_cluster.labels_

In [ ]:
stock_clusters = pd.DataFrame( { "StockCode" : cust_stocks_df.index,
                                 "ClusterID": h_cluster.labels_ } )

In [ ]:
stock_clusters

In [ ]:
stock_desc_dict = dict(zip(retail_df['StockCode'], retail_df['Description']))

In [ ]:
stock_clusters['Description'] = stock_clusters.StockCode.map(lambda x: stock_desc_dict[x])

In [ ]:
stock_clusters[stock_clusters.ClusterID == 0]

In [ ]:
stock_clusters[stock_clusters.ClusterID == 1]

In [ ]:
stock_clusters[stock_clusters.ClusterID == 2]

In [ ]:
stock_clusters[stock_clusters.ClusterID == 10]

In [ ]:
stock_clusters[stock_clusters.ClusterID == 12]